In [ ]:
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F

#import snowflake.snowpark

def initiateSession():
    connection_parameters = {
        "account": "mszvisn-web31075",
        "user": "jenkins_srv",
        "password": "Jenkins123",
        "role": "accountadmin",
        "warehouse": "compute_wh",
        "database":"snowpark_db",
        "schema":"cat_bronze"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

Transforming data from Bronze layer to Silver

In [ ]:
df_parts_raw = session.table("SNOWPARK_DB.CAT_BRONZE.PARTS")

In [ ]:
df_parts_raw.columns

In [ ]:
from snowflake.snowpark.types import IntegerType, FloatType

In [ ]:
udf = F.udf

In [ ]:
# Define and register the UDF using a decorator
#@udf(name='apply_discount', return_type=FloatType(), input_types=[FloatType()])
def apply_discount(cost: float) -> float:
    return cost * 0.9  # Apply 10% discount

#Register the UDF as a temporary function
apply_discount_udf = session.udf.register(apply_discount, return_type=FloatType(), input_types=[FloatType()])

Reformart the dates to proper date values. Then use UDF created to apply 10% discount on COST.

In [ ]:
df_parts_raw = df_parts_raw.withColumn("TRANSACTIONDATE", F.to_date(df_parts_raw["TRANSACTIONDATE"], 'DY MON DD, YYYY')) \
.withColumn("DELIVERYDATE", F.to_date(df_parts_raw["DELIVERYDATE"], 'DY MON DD, YYYY')) \
.withColumn("DISCOUNTED_COST", apply_discount(df_parts_raw["COST"]))

In [ ]:
df_parts_raw.show()

In [ ]:
df_parts_raw.write.mode("overwrite").save_as_table("snowpark_db.cat_silver.parts")